<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 3



### 4.1 Multiple Linear Regression Prediction



$$
\mathbb{y}_{n \times 1} = \mathbb{X}_{n \times m}\mathbb{w}_{m \times 1} + b_{-1\times 1}
$$



Here, -1 of $b_{-1 \times 1}$ means the `numpy` broadcasting.



#### Linear Class



In [ ]:
import torch.nn



In [ ]:
torch.manual_seed(1)
model = torch.nn.Linear(in_features=2, out_features=1)

list(model.parameters())



In [ ]:
model.state_dict()



Forward


In [ ]:
X = torch.tensor([1.0, 3.0])
yhat = model(X)
yhat



#### Custom Modules



In [ ]:
import torch.nn


torch.manual_seed(1)
class LR(torch.nn.Module):

  def __init__(self, input_size, output_size):
    super(LR, self).__init__()
    self.linear = torch.nn.Linear(input_size, output_size)

  def forward(self, x):
    return self.linear(x)

model = LR(input_size=2, output_size=1)



In [ ]:
X = torch.tensor(
    [
        [1.0, 1.0],
        [1.0, 2.0],
        [1.0, 3.0],
    ]
)
X


In [ ]:
model(X)



### 4.2 Multiple Linear Regression Training



$$
l\left(\mathbb{w}, b\right)=
\frac{1}{N}
\sum_{i=1}^N{
  \left(
    y_i - (x \cdot \mathbb{w}+b)
  \right)^2
}
$$



Gradient vector



$$
\nabla l(\mathbb{w}, b) = \begin{bmatrix}
  \frac{\partial}{\partial w_1}l(\mathbb{w}, b) \\
  \frac{\partial}{\partial w_2}l(\mathbb{w}, b) \\
  \vdots \\
  \frac{\partial}{\partial w_d}l(\mathbb{w}, b) \\
  \frac{\partial}{\partial b}l(\mathbb{w}, b) \\
\end{bmatrix}
$$

